In [1]:
from datasets import load_dataset
import random
random.seed(42)

sample_size_train = 133166
sample_size_test = 1268

dataset = load_dataset("Angelectronic/PhoMT_MTet")
dataset['test'] = dataset['test'].filter(lambda x: x['source'] == 'PhoMT')
# random_indices_train = random.sample(range(len(dataset['train'])), sample_size_train)
# dataset['train'] = dataset['train'].select(random_indices_train)
# dataset['train'] = dataset['train'].filter(lambda x: len(x['translation']['vi'].strip()) > 0 and len(x['translation']['en'].strip()) > 0)

# random_indices_test = random.sample(range(len(dataset['test'])), sample_size_test)
# dataset['test'] = dataset['test'].select(random_indices_test)
dataset

Found cached dataset parquet (C:/Users/lg/.cache/huggingface/datasets/Angelectronic___parquet/Angelectronic--PhoMT_MTet-4fb60bd61bf905c1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\lg\.cache\huggingface\datasets\Angelectronic___parquet\Angelectronic--PhoMT_MTet-4fb60bd61bf905c1\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-e17b29c78123ce07.arrow


DatasetDict({
    train: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 6280518
    })
    test: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 19111
    })
    dev: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 18656
    })
})

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig

base_model = "VietAI/envit5-translation"
adapter_model = "Angelectronic/iai-T5"

model = AutoModelForSeq2SeqLM.from_pretrained(base_model)
model = PeftModel.from_pretrained(model, adapter_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)

model = model.to("cuda")

c:\Users\lg\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
import evaluate
from tqdm import tqdm

metric = evaluate.load("sacrebleu")
labels = ['vi: ' + dataset['test'][i]['vi'] for i in range(len(dataset['test']))]
inputs = ['en: ' + dataset['test'][i]['en'] for i in range(len(dataset['test']))]
outputs = []
for input in tqdm(inputs):
    output = model.generate(tokenizer(input, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=1024, num_beams=5, num_return_sequences=1)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    outputs.append(output)

results = metric.compute(predictions=outputs, references=labels)
results

100%|██████████| 19111/19111 [12:22:50<00:00,  2.33s/it]  


{'score': 44.840040999922415,
 'counts': [304396, 207969, 145932, 107831],
 'totals': [425445, 406334, 387223, 368112],
 'precisions': [71.54767361233532,
  51.18178641216339,
  37.68681095905977,
  29.292986917025253],
 'bp': 1.0,
 'sys_len': 425445,
 'ref_len': 417194}